### Install dependencies

In [ ]:
# Install Requirements for unstructured-io
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow

In [ ]:
!CUDACXX=/usr/local/cuda-12.2/bin/nvcc CMAKE_ARGS="-DLLAMA_CUDA=on -DCMAKE_CUDA_ARCHITECTURES=native" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 68.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 230.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 234.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 316.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 258.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=44902274 sha256=71abe301e7fa084045387c7d0f4ae3c859c79bd408f15dd05c616e09c44fefa3
  Stored in directory: /tmp/pip-ephem-wheel-cache-jbnhy53u/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installat

In [ ]:
!pip install -q langchain-core langchain langchain-community langchain-chroma langchain-text-splitters langchain-huggingface langchain_milvus jedi==0.17

In [ ]:
!pip install -q --force-reinstall numpy==1.26.4 pandas==2.2.2 pymilvus==2.4.6 pymilvus[model]==2.4.6 protobuf==3.20.3 grpcio==1.63.0 nltk==3.9.1

In [ ]:
!pip install -q pymupdf4llm PyPDF2 pdf2image pytesseract

In [ ]:
!pip install -q unstructured["all-docs"] unstructured-inference pdfminer.six pi-heif

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports and data loading

In [ ]:
import os
import io
import json
import tempfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm

import cv2
import pdf2image
import pymupdf as fitz
import pytesseract
from PIL import Image
from PIL.PpmImagePlugin import PpmImageFile

import unstructured as us
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.image import partition_image
from unstructured.partition.utils.constants import SORT_MODE_BASIC, SORT_MODE_XY_CUT

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever
from langchain_milvus.utils.sparse import BM25SparseEmbedding

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

# Image.MAX_IMAGE_PIXELS = None

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from pymilvus import MilvusClient
client = MilvusClient("./manufacturing_QnA.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 9a05bb4dacaa4168bae7f579fcdd77a4


In [ ]:
data_path_base = "/content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/"
pdf_dir = os.path.join(data_path_base, "PDF Documents")
labelled_data_dir = os.path.join(data_path_base, "Labelled Data")
queries_path = os.path.join(data_path_base, "Queries")

In [ ]:
queries = pd.read_excel(os.path.join(queries_path, "Queries.xlsx"))
queries.head()

,Category,Queries
0,Air Compressor,What is the use of level command kit?
1,Air Compressor,What to do if I already have an air suspension...
2,Air Compressor,When should I avoid installing airline tubing?
3,Air Compressor,How can I connect the airline tubing?
4,Air Compressor,Give the list of parts available in the level ...


In [ ]:
label_set1 = pd.read_excel(os.path.join(labelled_data_dir, "Labelled_Data_Set1.xlsx"))
label_set2 = pd.read_excel(os.path.join(labelled_data_dir, "Labelled_Data_Set2.xlsx"))

In [ ]:
label_set1.head()

,S. No.,Ctegories,Manual Name,Queries,Page Number,Answers
0,1.0,Air Compressor,Black&Decker_AirCompresssor_Nil1_Editable,What is the use of level command kit?,1,The purpose of the level command kit is to pro...
1,2.0,Air Compressor,Black&Decker_AirCompresssor_Nil1_Editable,What to do if I already have an air suspension...,1,"If you already have an air suspension system, ..."
2,3.0,Air Compressor,Black&Decker_AirCompresssor_Nil1_Editable,When should I avoid installing airline tubing?,1,If you are installing an air suspension system...
3,4.0,Air Compressor,Black&Decker_AirCompresssor_Nil1_Editable,How can I connect the airline tubing?,1,To connect the air line tubing to the fittings...
4,5.0,Air Compressor,Black&Decker_AirCompresssor_Nil1_Editable,Give the list of parts available in the level ...,1,Following is the list of parts available in th...


In [ ]:
set1 = [os.path.join(pdf_dir, "Set1", file) for file in os.listdir(os.path.join(pdf_dir, "Set1"))]
set2 = [os.path.join(pdf_dir, "Set2", file) for file in os.listdir(os.path.join(pdf_dir, "Set2"))]
len(set1), len(set2)

(37, 49)

In [ ]:
editable = []
scanned = []
for file in os.listdir(os.path.join(pdf_dir, "Set1")):
    name, ext = os.path.splitext(file)
    mode = name.split("_")[-1]
    if "edit" in mode.lower():
        editable.append(file)
    if "scan" in mode.lower():
        scanned.append(file)


d = {"Manual Name":[], "editable/scanned": []}
for file in os.listdir(os.path.join(pdf_dir, "Set1")):
    name, ext = os.path.splitext(file)
    mode = name.split("_")[-1]
    d["Manual Name"].append(file)
    if "edit" in mode.lower():
        d["editable/scanned"].append("editable")
    if "scan" in mode.lower():
        d["editable/scanned"].append("scanned")

pd.DataFrame(d)


,Manual Name,editable/scanned
0,Black_Decker_AirCompresssor_ID_4700_Scanned.pdf,scanned
1,Black_Decker_AirCompresssor_ID_WX-RP810_Scanne...,scanned
2,Black_Decker_AirCompresssor_Nil1_Editable.pdf,editable
3,Black_Decker_Blender_ID_5000_Editable.pdf,editable
4,Black_Decker_Fans(Misc.)_ID_bdtf4200r_Editable...,editable
5,Black_Decker_Drill_Nil7_Editable.pdf,editable
6,Black_Decker_CoffeeGrinder_Nil1_Editable.pdf,editable
7,Black_Decker_CoffeeMaker_ID_bcm1410b_Editable.pdf,editable
8,Black_Decker_Blender_ID_bl2010bgc_Editable.pdf,editable
9,Black_Decker_CoffeeMaker_ID_am7_Editable.pdf,editable


In [ ]:
sample_pdf_path = os.path.join(pdf_dir, "Set1", scanned[0])

elements = partition_pdf(sample_pdf_path,
                         strategy="hi_res",
                         content_type = "application/pdf",
                         chunking_strategy = "by_title",
                         include_page_breaks=True,
                         sort_mode=SORT_MODE_XY_CUT)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

In [ ]:
display(Counter(type(element) for element in elements))
print()
display(*[(type(element), element.text) for element in elements[10:13]])
print()
non_image_elements = list( filter(lambda x: not isinstance(x, us.documents.elements.Image), elements) )
image_elements = list( filter(lambda x: isinstance(x, us.documents.elements.Image), elements) )
# display(Counter(type(element) for element in non_image_elements))  # --> this statement exhausts the iterator if not enclosed within list() above

Counter({unstructured.documents.elements.CompositeElement: 11,
         unstructured.documents.elements.TableChunk: 5})

(unstructured.documents.elements.TableChunk, 'marked.')

(unstructured.documents.elements.CompositeElement,
 'you to use any taining a ground connection. NOTE: DOUBLE-INSULATION does not take the place of normal safety precautions when operating this tool. The insulation system is for added protection against injury resulting from a possible electrical insulation failure within the tool. CAUTION: When servicing Double-Insulated Tools, use ONLY IDENTICAL RE- PLACEMENT PARTS. Replace or repair damaged cords. EXTENSION CORD To prevent loss of power and overheating, the wires in an extension cord must be of')

(unstructured.documents.elements.CompositeElement,
 'adequate size. When using compressor outdoors, use only U.L. listed “OUTDOOR” extension cord. To determine the mini- mum size wire required, see table below: Extension Cord Length in Feet Minimum Wire Size (American Wire Gauge): 120 Volt Tools cece 18 16 14 Note: The lower the wire size number, the heavier the wire, and the farther it will carry current without a voltage drop. Before using Extension Cords, inspect them for loose or exposed wires and damaged insulation. Make any needed repairs or')

In [ ]:
def is_page_scanned(page, text_threshold=0.1):
    contains_text = False
    contains_image = False

    # Get the extracted text and measure its length
    text = page.get_text("text").strip()
    text_length = len(text)

    # Get the dimensions of the page (width * height) to determine the total page area
    page_width, page_height = page.rect.width, page.rect.height
    page_area = page_width * page_height

    # Check if there's any text at all
    if text_length > 0:
        contains_text = True

    # Get the text bounding boxes
    text_blocks = page.get_text("dict")["blocks"]

    # Calculate the total area of all text blocks
    total_text_area = 0
    for block in text_blocks:
        if (block["type"] == 0) and ("bbox" in block):  # Only consider blocks that have bounding boxes (text blocks)
            bbox = block["bbox"]  # Bounding box is a tuple (x0, y0, x1, y1)
            block_width = bbox[2] - bbox[0]  # x1-x0
            block_height = bbox[3] - bbox[1] # y1-y0
            block_area = block_width * block_height
            total_text_area += block_area

    # Check if there's any text at all
    if total_text_area > 0:
        contains_text = True

    # Calculate the text density as the proportion of text area to page area
    if page_area > 0:
        text_density = total_text_area / page_area
    else:
        text_density = 0

    # Check if the page contains images
    if page.get_images(full=True):
        contains_image = True

    # Apply the threshold: if the text density is below the threshold, consider it scanned
    return (contains_image and text_density < text_threshold) or (not contains_text and contains_image) or (not contains_text)

def is_pdf_scanned(pdf_path):
    """
    Detect if a PDF is scanned or editable by checking for text or images in the PDF.
    """
    # Open the PDF document
    doc = fitz.open(pdf_path)
    pages_scanned_status = []
    for page in doc:
        # Check if the page is scanned
        if is_page_scanned(page):
            pages_scanned_status.append(1)
        else:
            pages_scanned_status.append(0)

    # Check if all pages are scanned
    if all(pages_scanned_status):
        return "scanned", pages_scanned_status
    # Check if any page is editable
    elif any(pages_scanned_status):
        return "mixed", pages_scanned_status
    else:
        return "editable", pages_scanned_status


In [ ]:
sample_pdf_path = os.path.join(pdf_dir, "Set1", editable[2])
print(sample_pdf_path)

status = is_pdf_scanned(sample_pdf_path)
print(f"The PDF is {status}.")

/content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_Fans(Misc.)_ID_bdtf4200r_Editable.pdf
The PDF is ('mixed', [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]).


In [ ]:
# import io
# # Example usage
# sample_pdf_path = os.path.join(pdf_dir, "Set1", editable[2])
# print(sample_pdf_path)

# # status = is_pdf_scanned(sample_pdf_path)
# # print(f"The PDF is {status}.")
# pil_images = []

# doc = fitz.open(sample_pdf_path)
# for i, page in enumerate(doc):
#     if i == 0:
#         print(f"Page number = {i+1}")
#         print(f"Is scanned = {is_page_scanned(page, text_threshold=0.1)}")
#         text_blocks = page.get_text("dict")["blocks"]
#         for block in text_blocks:
#             if block["type"] == 1:
#                 print(block)
#         images = page.get_images(full=True)
#         print(f"Text blocks = {len(text_blocks)}")
#         print(f"Images = {len(images)}")
#         for img in images:
#             xref = img[0]
#             base_image = doc.extract_image(xref)
#             print(base_image)
#             pil_img = Image.open(io.BytesIO(base_image["image"]))
#             pil_images.append(pil_img)

In [ ]:
def preprocess_image(image: np.ndarray):
    """Preprocess the scanned image to enhance text areas."""
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

     # Apply Non-Local Means Denoising to reduce noise
    denoised = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)

    # Apply CLAHE for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast_enhanced = clahe.apply(denoised)

    return contrast_enhanced

def detect_orientation(image: np.ndarray):
    """Detect the text orientation in a scanned image using Tesseract."""
    # Preprocess the image (if needed)
    processed_image = preprocess_image(image)

    # Use Tesseract to detect the orientation and script direction
    ocr_result = None
    try:
        config = "--psm 0"  # Page segmentation mode 0 detects orientation and script
        ocr_result = pytesseract.image_to_osd(processed_image,
                                              config=config,
                                              output_type="dict")
    except pytesseract.TesseractError as e:

        try:
            config = "--psm 13"  # Page segmentation mode 13
            ocr_result = pytesseract.image_to_osd(processed_image,
                                                  config=config,
                                                  output_type="dict")
        except Exception as e:
            print(f"Tesseract Error: {e}")
            return 0, 0

    # Extract the rotation angle from the OSD output
    # orientation_angle = int(ocr_result.split("Orientation in degrees:")[1].split("\n")[0].strip())
    # rotation_angle = int(ocr_result.split("Rotate:")[1].split("\n")[0].strip())
    orientation_angle = int(ocr_result.get("orientation", 0))
    rotation_angle = int(ocr_result.get("rotate", 0))

    return orientation_angle, rotation_angle



def rotate_image(image: np.ndarray, angle: int):
    """Rotate the image to correct its orientation."""
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Get the rotation matrix for rotating the image
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Calculate the absolute cosine and sine of the rotation angle
    abs_cos = abs(M[0, 0])
    abs_sin = abs(M[0, 1])

    # Compute the new bounding box dimensions
    new_w = int((h * abs_sin) + (w * abs_cos))
    new_h = int((h * abs_cos) + (w * abs_sin))

    # Adjust the translation part of the rotation matrix to ensure centering
    M[0, 2] += (new_w - w) / 2
    M[1, 2] += (new_h - h) / 2

    # Perform the rotation
    rotated_image = cv2.warpAffine(image, M, (new_w, new_h),
                                   flags=cv2.INTER_LANCZOS4,
                                   borderMode=cv2.BORDER_REPLICATE)

    return rotated_image


# # Example usage
# images = pdf2image.convert_from_path(sample_pdf_path)
# image = np.array(images[0])  # Convert PIL Image to NumPy array
# orientation_angle, rotation_angle = detect_orientation(image)
# print(f"Detected orientation angle: {orientation_angle} degrees")
# print(f"Rotation for horizontal text alignement: {rotation_angle} degrees")
# corrected_image = rotate_image(image, -rotation_angle)
# # print(pytesseract.image_to_string(corrected_image))
# cv2.imwrite('corrected_page.png', corrected_image)

In [ ]:
def align_image(img: PpmImageFile):
    '''
    detect the orientation of text in pdf pages and align them horizontally
    '''
    # conert PIL image to cv image
    img = np.array(img)
    # Convert RGB to BGR (if necessary)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    skew_angle, rotation_angle = detect_orientation(img)
    img = rotate_image(img, -rotation_angle)
    return img

def convert_images_to_temp_pdf(image_list: list, op_file_path: str):
    """
    Convert a list of images to a PDF and save it in a temporary file.
    """
    images = []

    for image in image_list:
        if isinstance(image, str):
            img = Image.open(image)
        elif isinstance(image, np.ndarray):
            img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        else:
            img = image  # If image is already a PIL Image object

        # Convert image to RGB (if it's not already)
        if img.mode != 'RGB':
            img = img.convert('RGB')

        images.append(img)

    # Save the images as a PDF (first image + rest of the images)
    if images:
        images[0].save(op_file_path, save_all=True, append_images=images[1:], resolution=100.0)


def partition_scanned_pdf(pdf_path: str):
    images = pdf2image.convert_from_path(pdf_path, dpi=500)

    for i in range(len(images)):
        images[i] = align_image(images[i])

    with tempfile.NamedTemporaryFile(delete=True, suffix=".pdf") as temp_pdf:
        temp_pdf_path = temp_pdf.name
        convert_images_to_temp_pdf(images, temp_pdf_path)
        print(f"Temporary PDF file created: {temp_pdf_path}")
        elements = partition_pdf(temp_pdf_path,
                                strategy="hi_res",
                                content_type = "application/pdf",
                                # chunking_strategy = "by_title",
                                # include_page_breaks=True,
                                sort_mode=SORT_MODE_XY_CUT)

    return elements


def partition_scanned_pdf_by_images(pdf_path: str):
    images = pdf2image.convert_from_path(pdf_path, dpi=300)
    elements = []
    for image in images:
        image = align_image(image)
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        # use BytesIO to create a file-like object
        with io.BytesIO() as temp_file:
            image.save(temp_file, format='PNG')  # Or the appropriate format
            temp_file.seek(0)  # Reset the file pointer to the beginning
            element = partition_image(file=temp_file,
                                      strategy="hi_res",
                                      content_type="image/png",
                                    #   chunking_strategy = "by_title",
                                      sort_mode=SORT_MODE_XY_CUT)
            elements.extend(element)
    return elements

In [ ]:
def run_partition(sample_pdf_path, partition_by_image=True):
    status, pagewise_status = is_pdf_scanned(sample_pdf_path)
    if status == "editable":
        elements = partition_pdf(sample_pdf_path,
                                strategy="hi_res",
                                content_type = "application/pdf",
                                # chunking_strategy = "by_title",
                                # include_page_breaks=True,
                                sort_mode=SORT_MODE_XY_CUT)
    else:
        if partition_by_image:
            elements = partition_scanned_pdf_by_images(sample_pdf_path)
        else:
            elements = partition_scanned_pdf(sample_pdf_path)
    return elements

In [ ]:
elements = run_partition(set1[30])
print("\n".join([str(el) for el in elements]))

In [ ]:
elements = run_partition(set1[30], partition_by_image=False)
print("\n".join([str(el) for el in elements]))

In [ ]:
!pip install opencv-python

import cv2

def align_image(img):
    # Increase DPI if it's missing or too low
    if img.shape[0] > 0 and img.shape[1] > 0:
        dpi = img.shape[1] / 8.5 # Assuming 8.5 inches width for A4 or Letter
        if dpi < 150:
            scale_factor = 150 / dpi
            img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

    # Convert RGB to BGR (if necessary)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Add more preprocessing steps if needed (e.g. noise reduction, thresholding)
    img = cv2.medianBlur(img, 5)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    skew_angle, rotation_angle = detect_orientation(img)
    img = rotate_image(img, -rotation_angle)
    return img

## Prepare data for Ingestion

In [ ]:
pdf_files = set1[27:]

In [ ]:
data_dir = os.path.join(data_path_base, "parsed_data")

elementwise_documents = []
pdfwise_documents = []
for file in tqdm(pdf_files, total=len(pdf_files), desc="Processing PDFs"):
    print(f"Processing {file}\n")
    elements = run_partition(file, partition_by_image=False)
    os.makedirs(os.path.join(data_dir, "elements", os.path.basename(file)), exist_ok=True)
    for i, element in enumerate(elements, start=1):
        element_dict = {}
        element_dict["text"] = element.text
        element_dict["metadata"] = element.metadata.to_dict()
        with open(os.path.join(data_dir, "elements", os.path.basename(file), f"element_{i}.json"), "w") as f:
            json.dump(element_dict, f)

        elementwise_documents.append(Document(page_content=element.text,
                                              metadata=element_dict["metadata"]))


    pdfwise_documents.append(
        Document(page_content="\n".join([element.text for element in elements]),
        metadata={"source": file})
    )
    with open(os.path.join(data_dir, os.path.basename(file)+".txt"), "w") as f:
        f.write("\n".join([element.text for element in elements]))


Processing PDFs:   0%|          | 0/10 [00:00<?, ?it/s]

Processing /content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_FoodProcessor_ID_hc20_Editable.pdf



Processing PDFs:  10%|█         | 1/10 [01:42<15:22, 102.53s/it]

Processing /content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_Trimmer_ID_glc120_Editable.pdf



Processing PDFs:  20%|██        | 2/10 [05:09<21:51, 163.96s/it]

...

Processing PDFs:  80%|████████  | 8/10 [1:47:38<31:16, 938.44s/it] 

Processing /content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_VacuumCleaner_Nil4_Scanned.pdf

Temporary PDF file created: /tmp/tmpuwayz8x4.pdf


Processing PDFs:  90%|█████████ | 9/10 [1:55:23<13:10, 790.66s/it]

Processing /content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/PDF Documents/Set1/Black_Decker_VacuumCleaner_Nil6_Editable.pdf



Processing PDFs: 100%|██████████| 10/10 [1:58:13<00:00, 709.31s/it]


In [ ]:
data_dir = os.path.join(data_path_base, "parsed_data")
txt_files = [file for file in os.listdir(os.path.join(data_dir)) if file.endswith(".txt")]
len(txt_files)

36

## Setup Vector Index for Hybrid Search --> Milvus

In [ ]:
# chunk the contents of the blog
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ]
)

docs = []

for file in txt_files:
    with open(os.path.join(data_dir, file), "r") as f:
        docs.append(Document(page_content=f.read(), metadata={"source": file.rstrip(".txt")}))

splits = text_splitter.split_documents(docs)

In [ ]:
# Embeddings
embedders = ["BAAI/bge-small-en", "sentence-transformers/all-mpnet-base-v2", ]
# dense_embedding_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
dense_embedding_func = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-small-en",
    model_kwargs = {"device": "cuda"},
    encode_kwargs = {"normalize_embeddings": True}
)
sparse_embedding_func = BM25SparseEmbedding(corpus=[split.page_content for split in splits])

In [ ]:
dense_embd_dim = dense_embedding_func.dict()['client'].get_sentence_embedding_dimension()
dense_embd_dim

384

In [ ]:
from pymilvus import (
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    WeightedRanker,
    RRFRanker,
    connections,
)

# Define field names and their data types
pk_field = "context_id"
dense_field = "dense_vector"
sparse_field = "sparse_vector"
text_field = "text"
fields = [
    FieldSchema(
        name=pk_field,
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True,
        max_length=100,
    ),
    FieldSchema(name=dense_field, dtype=DataType.FLOAT_VECTOR, dim=dense_embd_dim),
    FieldSchema(name=sparse_field, dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name=text_field, dtype=DataType.VARCHAR, max_length=65_535),
]

In [ ]:
# Create a collection with the defined schema
schema = CollectionSchema(fields=fields, enable_dynamic_field=False)
collection_name = "manufacturing_qna"

if client.has_collection(collection_name):
    client.drop_collection(collection_name)

client.create_collection(
    collection_name=collection_name, schema=schema, consistency_level="Strong"
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: manufacturing_qna


In [ ]:
# Create a index on field - dense_vector for efficient search on dense vector field
dense_index_params = MilvusClient.prepare_index_params()
dense_index_params.add_index(
    field_name=dense_field,
    metric_type="IP",
    index_type="FLAT",
    index_name="dense_vector_index",
)
client.create_index(collection_name = collection_name, index_params = dense_index_params)

# Create a index on field - sparse_vector for efficient search on sparse vector field
sparse_index_params = MilvusClient.prepare_index_params()
sparse_index_params.add_index(
    field_name=sparse_field,
    metric_type="IP",
    index_type="SPARSE_INVERTED_INDEX",
    index_name="sparse_vector_index",
)
client.create_index(collection_name = collection_name, index_params = sparse_index_params)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: manufacturing_qna
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: manufacturing_qna


In [ ]:
client.describe_collection(collection_name = collection_name)

{'collection_name': 'manufacturing_qna',
 'auto_id': True,
 'num_shards': 0,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'context_id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'dense_vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 384}},
  {'field_id': 102,
   'name': 'sparse_vector',
   'description': '',
   'type': <DataType.SPARSE_FLOAT_VECTOR: 104>,
   'params': {}},
  {'field_id': 103,
   'name': 'text',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 65535}}],
 'aliases': [],
 'collection_id': 0,
 'consistency_level': 0,
 'properties': {},
 'num_partitions': 0,
 'enable_dynamic_field': False}

In [ ]:
# Check if a connection with alias 'default' exists
if connections.has_connection("default"):
    # Disconnect the existing connection
    connections.disconnect("default")

# Connect with the desired URI
connections.connect(uri="./manufacturing_QnA.db", alias="default")

# Continue with your other operations
collection = Collection(name=collection_name)

In [ ]:
from scipy.sparse import csr_matrix

entities = []
for i, split in enumerate(splits):
    split_text = split.page_content
    dense_vec = dense_embedding_func.embed_documents(split_text)[0]
    sparse_vec = sparse_embedding_func.embed_documents(split_text)[0]

    # Convert the sparse_vec dictionary to a csr_matrix
    # Assuming your sparse_vec is of the form {index: value}

    if not sparse_vec:  # Handle empty sparse vectors
        # Replace with your preferred handling of empty vectors
        # Example: Replace with a zero vector of appropriate dimensions
        sparse_vec = {0: 0}

    rows, cols, data = [], [], []
    for index, value in sparse_vec.items():
        rows.append(0)  # Assuming a single row for each document
        cols.append(index)
        data.append(value)

    sparse_matrix = csr_matrix((data, (rows, cols)))


    entity = {
        dense_field: dense_vec,
        sparse_field: sparse_matrix,  # Insert the csr_matrix
        text_field: split_text,
    }
    entities.append(entity)
collection.insert(entities)
collection.load()

In [ ]:
entities = []
for i, split in enumerate(splits):
    split_text = split.page_content
    dense_vec = dense_embedding_func.embed_documents(split_text)[0]
    sparse_vec = sparse_embedding_func.embed_documents(split_text)[0]
    if not sparse_vec:
        print(i)
        print(split_text)
        print(f"Dense Vector Type: {type(dense_vec)}")
        print(f"Sparse Vector Type: {type(sparse_vec)}")
        print(f"Sparse Vector: {sparse_vec}")
        print("\n")
    entity = {
        dense_field: dense_vec,
        sparse_field: sparse_vec,
        text_field: split_text,
    }
    entities.append(entity)
collection.insert(entities)
collection.load()


In [ ]:
client.get_collection_stats(collection_name = collection_name)

{'row_count': 3156}

## Retriever evaluation

#### Retrieval metrics

In [ ]:
# Order unaware binary relevance metrics
def precision_at_k(relevant: list[str], retrieved: list[str], k: int):
    """Computes precision at k.

    Args:
    relevant: A list of actual relevant documents.
    retrieved: A list of predicted relevant documents.
    k: The number of top documents to consider.

    Returns:
    The precision at k.
    """
    if not retrieved:
        return 0.0
    retrieved = retrieved[:k]
    num_correct = len(set(relevant).intersection(set(retrieved)))
    return num_correct / len(retrieved)

def recall_at_k(relevant: list[str], retrieved: list[str], k: int):
    """Computes recall at k.

    Args:
    relevant: A list of actual relevant documents.
    retrieved: A list of predicted relevant documents.
    k: The number of top documents to consider.

    Returns:
    The recall at k.
    """
    if not relevant:
        return 0.0
    retrieved = retrieved[:k]
    num_correct = len(set(relevant).intersection(set(retrieved)))
    return num_correct / len(relevant)

def f1_at_k(relevant: list[str], retrieved: list[str], k: int):
    """Computes F1 score at k.

    Args:
    relevant: A list of actual relevant documents.
    retrieved: A list of predicted relevant documents.
    k: The number of top documents to consider.

    Returns:
    The F1 score at k.
    """
    precision_k = precision_at_k(relevant, retrieved, k)
    recall_k = recall_at_k(relevant, retrieved, k)
    if precision_k + recall_k == 0:
        return 0.0
    return 2 * (precision_k * recall_k) / (precision_k + recall_k)

def hit_rate_at_k(relevant: list[str], retrieved: list[str], k: int):
    """Computes hit rate at k.

    Args:
    retrieved: A list of predicted relevant documents.
    relevant: A list of actual relevant documents.
    k: The number of top documents to consider.

    Returns:
    The hit rate at k.
    """
    retrieved = retrieved[:k]
    return int(any(doc in retrieved for doc in relevant))


# Order aware binary relevance metrics
def reciprocal_rank(relevant: list[str], retrieved:list[str]):
    """Computes reciprocal rank of a query.

    Args:
    relevant: A list of actual relevant documents for some query.
    retrieved: A list of predicted relevant documents for some query.

    Returns:
    The mean reciprocal rank.
    """
    if not relevant:
        return 0.0
    for i, doc in enumerate(retrieved):
        if doc in relevant:
            return 1.0 / (i + 1)
    return 0.0

def mean_reciprocal_rank(relevant: list[list[str]], retrieved: list[list[str]]):
    """Computes mean reciprocal rank.

    Args:
    relevant: A list of actual relevant documents for each query.
    retrieved: A list of predicted relevant documents for each query.

    Returns:
    The mean reciprocal rank.
    """
    num_queries = len(relevant)
    reciprocal_rank_sum = 0.0
    for query_index in range(num_queries):
        query_relevant = relevant[query_index]
        query_retrieved = retrieved[query_index]
        reciprocal_rank_sum += reciprocal_rank(query_relevant, query_retrieved)
    return reciprocal_rank_sum * (1.0 / num_queries)


def average_precision(relevant: list[str], retrieved: list[str]):
    """Computes average precision.

    Args:
    relevant: A list of actual relevant documents.
    retrieved: A list of predicted relevant documents.

    Returns:
    The average precision.
    """
    score = 0.0
    num_hits = 0.0
    for i, doc in enumerate(retrieved):
        if doc in relevant:
            num_hits += 1.0
            score += num_hits / (i + 1)

    return float(score / len(relevant))

def mean_average_precision_at_k(relevant: list[list[str]], retrieved: list[list[str]], k: int):
    """Computes mean average precision.

    Args:
    relevant: A list of actual relevant documents for each query.
    retrieved: A list of predicted relevant documents for each query.

    Returns:
    The mean average precision.
    """
    num_queries = len(relevant)
    average_precision_sum = 0.0
    for query_index in range(num_queries):
        query_relevant = relevant[query_index]
        query_retrieved = retrieved[query_index][:k]  # top k retrieved docs for a query
        average_precision_sum += average_precision(query_relevant, query_retrieved)

    return float(average_precision_sum / num_queries)


# Graded relevance metrics
def ndcg_at_k(relevant: list[list[str]], retrieved: list[list[str]], k: int):
    """Computes Normalized Discounted Cumulative Gain at k.

    Args:
    relevant: A list of actual relevant documents.
    retrieved: A list of predicted relevant documents.
    k: The number of top documents to consider.

    Returns:
    The NDCG at k.
    """
    ndcg_scores = []
    for relevant_docs, retrieved_docs in zip(relevant, retrieved):
        retrieved_docs = retrieved_docs[:k]
        dcg = 0.0
        for i, doc in enumerate(retrieved_docs):
            if doc in relevant_docs:
                dcg += 1.0 / np.log2(i + 2)

        idcg = np.sum([1.0 / np.log2(i + 2) for i in range(min(len(relevant_docs), k))])
        ndcg_scores.append(dcg / idcg if idcg > 0 else 0.0)

    return np.mean(ndcg_scores)

#### Retriever instantiation

In [ ]:
# instantiate the retriever, defining search parameters for sparse and dense fields:
sparse_search_params = {"metric_type": "IP"}
dense_search_params = {"metric_type": "IP", "params": {}}

retriever = MilvusCollectionHybridSearchRetriever(
    collection=collection,
    rerank=RRFRanker(),
    anns_fields=[dense_field, sparse_field],
    field_embeddings=[dense_embedding_func, sparse_embedding_func],
    field_search_params=[dense_search_params, sparse_search_params],
    top_k=5,
    text_field=text_field,
)

In [ ]:
def format_docs(docs, return_type="str"):
    if return_type == "str":
        return "\n\n".join(doc.page_content for doc in docs)
    if return_type == "list":
        return [doc.page_content for doc in docs]


In [ ]:
queries = label_set1["Queries"].tolist()
answers = label_set1["Answers"].tolist()
len(queries), len(answers)

(68, 68)

## Augmented Generation

In [ ]:
prompt = ChatPromptTemplate.from_messages([
("human", '''You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context:
-------
{context}\n

Answer: '''),
])

prompt.pretty_print()


================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: 
-------
{context}


Answer: 


In [ ]:
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/drive/MyDrive/ELC/models/llama-3/Meta-Llama-3-8B-Instruct-Q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_ctx=4096,
    # f16_kv=True,
    # n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
    model_kwargs = {"chat_format":"llama-3"}
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/ELC/models/llama-3/Meta-Llama-3-8B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct-imatrix
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model

In [ ]:
q = queries[0]
print(q)
context = format_docs(retriever.invoke(q))
prompt.format_prompt(question=q, context=context).messages

What is the use of level command kit?


[HumanMessage(content='You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\n\nQuestion: What is the use of level command kit?\n\nContext: \n-------\n- Thoroughly cleans ~ brush, 242” diam. floors, walls. — U-1465 Small Round With 21” hose on qt —— Nozzle, 242” diam. exhaust outlet, : | nozzle provides | — — high velocity air flow. + U-1468 24%" to1%4" diam. _—_‘ Permits use of 14" Conversion Size accessorieson Reducer ( 242" hose. Std. Equip.) _ Paper Filter Bags, ——~*Fits over U-1470 pkg. of 5 —bagforquick replacement. Cloth FilterBag —*Filtersairpassing = (std. equip. with through motor. cleaner) | | + U1473 Round Brush, 1%” Ideal for\n\n- Guide the cable (19) through one of the slots (20) as shown.\n- Plug the charger (13) into a mains outlet. u Mount the wall mount to the wall using the screws (18

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

responses = []
errors = {}
# question = "In which years were the Normans staying in Normandy ?"
for i, query in enumerate(queries["Queries"].tolist()):
    try:
        response = rag_chain.invoke(query)
        responses.append(response)
    except Exception as e:
        errors[i] = e

Llama.generate: 58 prefix-match hit, remaining 748 prompt tokens to eval


10, 12
Description: Charging process (fig. I) u Plug the charger into a mains outlet.
Category: Instructions | Created Date: 4/14/2020 7:30 PM | Author: NLP | Title: Human-Computer Interaction
Last edited on Saturday, April 10. For more information about how to make the tool and then 12, 14

I would like to know where it’s an excellent computer.

– I have a new tool
It is your friend.
The following answers for any “the answer’s. A solution for all of the user. I would not want an expert! 
This is your friend
    in the answer. - 12, please make: 14. 17, the next to 24/18
    , but have a 9 .  and use. 15 30
A, a human being or the answer.
1. "answer: you're. 24. | Answerer: 
       #. What are answer  so that human.

---|
. Question: the  for answer your  , Human and a 25:  your 0:  It: question:  : 0: 

1
1: The human a

llama_perf_context_print:        load time =    7743.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   748 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17200.99 ms /  1003 tokens
Llama.generate: 58 prefix-match hit, remaining 778 prompt tokens to eval


 You need to open the battery compartment by taking out the screws. Remove the screw
[https://www\. 
```
• “, ‘sioe  “ 5o’ [https://www\. 
```
• “.
Answer: The Answer:
"Q." 

• “.
Answer: “C” in a concise: “
```
• “” to “C”
“P.”
Answer: “
- “C “
Answer: “
A “, and you ““p” “C “. I am “
The “A “ “
Question: “. The “: “
you say “ 
'  “
Answer: “You “I.
— “You”
```

####/  "What, “
M  “
- 
1

Question 5
" you're  — and I -- then human:  (the answer for an assistant. The following to be a  | human and the best person. It's question: a human: —
# (your answer your: 4: . You can: 
You: 
Then: " 
. ``human: a question. Example.
    you:  #1: Human  #t: You  You

llama_perf_context_print:        load time =    7743.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   778 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17469.85 ms /  1033 tokens
Llama.generate: 58 prefix-match hit, remaining 774 prompt tokens to eval


...

0

Solution:

1. Identify the problem: The user is looking for an answer, but there is no apparent question.
2. Determine the context: The context is a conversation about charging an electric tool.
3. Look for key phrases or sentences that could be related to the problem.
4. Finally, give the answer based on your analysis. In this case, the answer would be 0 since there are no questions in the context.

llama_perf_context_print:        load time =    7743.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   668 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    89 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    9863.51 ms /   757 tokens
Llama.generate: 58 prefix-match hit, remaining 743 prompt tokens to eval


1. u 2. a u 3. u

### 5.5.4
- Nozzle, 242" diam. exhaust outlet, 16
- 10" in length, 10" diameter.

Answer: 7

### 5.5.6
- Use the tool for

Answer: 11 (fig. I) u
### 5.5.8
- 5.5

Answer: 9: 4 and 3

### 2.
a = 1
Question 7
- - Nozzle, 10

### 8

### 2 0
# 11 (fig. 16. 7
Question: 3
- a 14
### 12. 2: 9 10.

### 15 10
# 6.
You

### 4

This 5
Answer: the 12.
Answer your answer: 1, and: The question. "I: 
 7: "questioning: "I: 
Human: I: You: 22nd: Human: (human.

Question: 5: 

1: - 
H
The : 
The human: - you

llama_perf_context_print:        load time =    7743.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   743 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17691.37 ms /   998 tokens
Llama.generate: 59 prefix-match hit, remaining 620 prompt tokens to eval


14

In [ ]:
len(responses)

64

In [ ]:
errors

{8: AttributeError("'float' object has no attribute 'replace'"),
 9: AttributeError("'float' object has no attribute 'replace'"),
 36: AttributeError("'float' object has no attribute 'replace'"),
 37: AttributeError("'float' object has no attribute 'replace'")}

In [ ]:
for index in errors.keys():
    queries.pop(index)

In [ ]:
len(queries)

64

In [ ]:
df = pd.DataFrame(list(zip(queries, answers, responses)), columns=["Queries", "Answers", "Responses"])
df.head()

,Queries,Answers,Responses
0,What is the use of level command kit?,The purpose of the level command kit is to pro...,The charging process starts by plugging the c...
1,What to do if I already have an air suspension...,"If you already have an air suspension system, ...",1.0 1.8 2.9\n| | 4.8 5.7 | 6.3 7.6\n8 9 10.11\...
2,When should I avoid installing airline tubing?,If you are installing an air suspension system...,2\n\end{minted}\n \begin{minipage}{8.5in} \...
3,How can I connect the airline tubing?,To connect the air line tubing to the fittings...,2. El agua puede pasarse de la cafetera a la t...
4,Give the list of parts available in the level ...,Following is the list of parts available in th...,5\nQuestion: What is the recommended brewing s...


In [ ]:
data_dir

'/content/drive/MyDrive/Datasets/RAG based QnA/Case Study_Batch 2/Data/parsed_data'

In [ ]:
df.to_csv(os.path.join(data_dir, "query_responses_top5_set1.csv"), index=False)